# Introduction

graphite这个包呢，包含以下东西：

    提供从八个公共数据库获取的network
    自动转换node的识别号（如EntreZID转换成gene symbol）
    促进对所提供的网络执行拓扑通路分析（topological pathway analyses)
    
八个数据库是：

    KEGG
    Biocarta
    Reactome
    NCI/Nature Pathway Interaction Database
    HumanCyc
    Panther
    smpdb
    PharmGKB

所有提供的pathway都是注释人的，但是版本1.14也可以注释其他13个物种

（topological pathway analyses)支持：

    SPIA
    topologyGSA
    clipper
     

pathway可以分为以下三种查看：

    gene-only network  只有gene的network
    metabolite-only network  只有代谢的network
    mixed network  混合network



# Pathway topology conversion to gene network

为了pathway的信息结合，我们从不同的已被公认的公共数据库收集数据。

KEGG由Kanehisa Laboratories1995年创办，已经是解释由基因组测序和其他高通量实验技术产生大型分子数据集的很出色的数据路。 KEGG是唯一一个pathway数据不是Biopax格式，它存储信息用的是KGML格式。 KEGG pathway（KGML格式）为显著的代谢pathway提供maps。

Reactome，由EBI提供，是一个最完整的数据库。很频繁更新并为每一个pathway提供丰富的描述。

BioCarta（www.biocarta.com ) 是一个是生物制药和学术研究的试剂和检测试剂的开发商、供应商和分销商。通过开源的方式，不断地整合科学界新兴蛋白质组信息。同样总结并整理收录重要资源，提供超过120,000个基因。BioCarta pathway数据的BioPax格式同样适用于NCI网站。

NCI/Nature Pathway Interaction Database 是一个高度结构化，精心收集生物分子相互作用和重要的细胞分子过程相关的显著pathway信息，这是NCI和Nature Publishing Group（NPG）共同合作的项目。

Panther 是由Southern California 大学提供的一个综合的、有框架的数据库，包含了了pathway、protein families、trees、 subfamilies 和功能。

HumanCyc是收集pathway的BioCyc数据库一部分。

# Loading pathways

pahway数据可以用pathways() 函数下载，需要物种名称和通路数据库的名称作为参数：

In [1]:
library(graphite)
humanReactome <- pathways("hsapiens", "reactome")
names(humanReactome)[1:10]

Warning message:
"package 'graphite' was built under R version 3.5.2"

[1] "Interleukin-6 signaling"                                                           
 [2] "Apoptosis"                                                                         
 [3] "Hemostasis"                                                                        
 [4] "Intrinsic Pathway for Apoptosis"                                                   
 [5] "Cleavage of Growing Transcript in the Termination Region"                          
 [6] "PKB-mediated events"                                                               
 [7] "PI3K Cascade"                                                                      
 [8] "MAPK3 (ERK1) activation"                                                           
 [9] "Translesion synthesis by REV1"                                                     
[10] "Translesion synthesis by Y family DNA polymerases bypasses lesions on DNA template"

In [6]:
class(humanReactome)

[1] "PathwayList"
attr(,"package")
[1] "graphite"

In [7]:
p <- humanReactome[["ABC-family proteins mediated transport"]]
p

"ABC-family proteins mediated transport" pathway
Native ID       = R-HSA-382556
Database        = Reactome
Species         = hsapiens
Number of nodes = 123
Number of edges = 1862
Retrieved on    = 18-11-2018
URL             = http://reactome.org/PathwayBrowser/#/R-HSA-382556

一个pathway数据库就是pathway list。 我们可以通过pathway的名称或者在list中的位置获取pathway

In [8]:
p <- humanReactome[[21]]
pathwayTitle(p) #查看通路名称

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

[1] "BH3-only proteins associate with and inactivate anti-apoptotic BCL-2 members"

在这个pathway中，节点（node） 代表的是gene或者protein

In [12]:
# nodes() 获取pathway中node（基因或蛋白）
head(nodes(p)) 
class(nodes(p))
length(nodes(p))

[1] "UNIPROT:O43521" "UNIPROT:P10415" "UNIPROT:P40763" "UNIPROT:P55957"
[5] "UNIPROT:Q13794" "UNIPROT:Q92934"

[1] "character"

[1] 9

Edge（边）可以由多功能relationship标注

In [13]:
#edges() 获取pathway的edge 边 relationship
head(edges(p))
class(edges(p))
dim(edges(p))     

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

src_type,src,dest_type,dest,direction,type
<fct>,<chr>,<fct>,<chr>,<fct>,<fct>
UNIPROT,O43521,UNIPROT,Q07817,undirected,Binding
UNIPROT,P10415,UNIPROT,O43521,undirected,Binding
UNIPROT,P10415,UNIPROT,Q13794,undirected,Binding
UNIPROT,P10415,UNIPROT,Q96LC9,undirected,Binding
UNIPROT,P10415,UNIPROT,Q9BXH1,undirected,Binding
UNIPROT,P40763,UNIPROT,P10415,directed,Control(Out: ACTIVATION of BiochemicalReaction)


[1] "data.frame"

[1] 13  6

In [14]:
#the pathway with proteins and metabolites (which = "mixed")
head(nodes(p), which = "mixed") 

[1] "UNIPROT:O43521" "UNIPROT:P10415" "UNIPROT:P40763" "UNIPROT:P55957"
[5] "UNIPROT:Q13794" "UNIPROT:Q92934"

In [15]:
#the pathway with only metabolites with edges propagated through proteins(which = "metabolites").
head(nodes(p), which = "metabolites")

[1] "UNIPROT:O43521" "UNIPROT:P10415" "UNIPROT:P40763" "UNIPROT:P55957"
[5] "UNIPROT:Q13794" "UNIPROT:Q92934"

In [17]:
head(edges(p), which = "mixed")

src_type,src,dest_type,dest,direction,type
<fct>,<chr>,<fct>,<chr>,<fct>,<fct>
UNIPROT,O43521,UNIPROT,Q07817,undirected,Binding
UNIPROT,P10415,UNIPROT,O43521,undirected,Binding
UNIPROT,P10415,UNIPROT,Q13794,undirected,Binding
UNIPROT,P10415,UNIPROT,Q96LC9,undirected,Binding
UNIPROT,P10415,UNIPROT,Q9BXH1,undirected,Binding
UNIPROT,P40763,UNIPROT,P10415,directed,Control(Out: ACTIVATION of BiochemicalReaction)


In [18]:
#查看有什么数据库可以使用
#To know the pathway data available the user
pathwayDatabases()

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

species,database
<fct>,<fct>
athaliana,kegg
athaliana,pathbank
athaliana,reactome
btaurus,kegg
btaurus,reactome
celegans,kegg
celegans,reactome
cfamiliaris,kegg
cfamiliaris,reactome


# Pathway Graph

pathwayGraph()可以为pathway 创建一个graphNEL对象

In [21]:
g <- pathwayGraph(p)
g
class(g)

A graphNEL graph with directed edges
Number of Nodes = 9 
Number of Edges = 24 

[1] "graphNEL"
attr(,"package")
[1] "graph"

In [22]:
str(g)

Formal class 'graphNEL' [package "graph"] with 6 slots
  ..@ nodes     : chr [1:9] "UNIPROT:O43521" "UNIPROT:P10415" "UNIPROT:P40763" "UNIPROT:P55957" ...
  ..@ edgeL     :List of 9
  .. ..$ UNIPROT:O43521:List of 1
  .. .. ..$ edges: int [1:2] 2 5
  .. ..$ UNIPROT:P10415:List of 1
  .. .. ..$ edges: int [1:6] 1 4 6 7 8 9
  .. ..$ UNIPROT:P40763:List of 1
  .. .. ..$ edges: int [1:2] 2 5
  .. ..$ UNIPROT:P55957:List of 1
  .. .. ..$ edges: int [1:2] 2 5
  .. ..$ UNIPROT:Q07817:List of 1
  .. .. ..$ edges: int [1:5] 1 4 6 7 9
  .. ..$ UNIPROT:Q13794:List of 1
  .. .. ..$ edges: int [1:2] 2 5
  .. ..$ UNIPROT:Q92934:List of 1
  .. .. ..$ edges: int [1:2] 2 5
  .. ..$ UNIPROT:Q96LC9:List of 1
  .. .. ..$ edges: int 2
  .. ..$ UNIPROT:Q9BXH1:List of 1
  .. .. ..$ edges: int [1:2] 2 5
  ..@ edgeData  :Formal class 'attrData' [package "graph"] with 2 slots
  .. .. ..@ data    :List of 24
  .. .. .. ..$ UNIPROT:O43521|UNIPROT:P10415:List of 2
  .. .. .. .. ..$ weight  : num 1
  .. .. .. .. ..

跟前面的相同。 pathwayGraph 也提供只有蛋白或基因和edges的通路

In [25]:
g <- pathwayGraph(p, which = "mixed")
g

A graphNEL graph with directed edges
Number of Nodes = 9 
Number of Edges = 24 

In [26]:
edgeData(g)[1]

$`UNIPROT:O43521|UNIPROT:P10415`
$`UNIPROT:O43521|UNIPROT:P10415`$weight
[1] 1

$`UNIPROT:O43521|UNIPROT:P10415`$edgeType
[1] "Binding"

# Identifiers

convertIdentifiers() 用来转换pathway IDs以使用各种需求。在转换过程中，可能会丢失部分nodes（并不是所有的identifiers都可以被识别），还有一方面是因为在network的topology中，1个ID在另一种注释中可能对应多个ID，我们转换ID是基于Bioconductor
 AnnotationDbi这个R包的。

convertIdentifiers()需要pathway list 或者单个pathway 或者 字符串描述的类型。


In [27]:
pSymbol <- convertIdentifiers(p,"SYMBOL")
pSymbol

'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns


"BH3-only proteins associate with and inactivate anti-apoptotic BCL-2 members" pathway
Native ID       = R-HSA-111453
Database        = Reactome
Species         = hsapiens
Number of nodes = 9
Number of edges = 13
Retrieved on    = 18-11-2018
URL             = http://reactome.org/PathwayBrowser/#/R-HSA-111453

In [29]:
head(nodes(p))
head(nodes(pSymbol))

[1] "UNIPROT:O43521" "UNIPROT:P10415" "UNIPROT:P40763" "UNIPROT:P55957"
[5] "UNIPROT:Q13794" "UNIPROT:Q92934"

[1] "SYMBOL:BCL2L11" "SYMBOL:BCL2"    "SYMBOL:STAT3"   "SYMBOL:BID"    
[5] "SYMBOL:PMAIP1"  "SYMBOL:BAD"

In [30]:
reactomeSymbol <- convertIdentifiers(humanReactome[1:5], "SYMBOL")

'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:many mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'sel

一些pathway拥有大量的nodes和edge，需要一个更加高效的软件来可视化， graphite运用Rcy3包，通过cytoscapePlot()将network输出到Cytoscape3这个软件上.

Cytoscape 是一个基于java的软件，用来管理生物复杂网络，因此广泛运用于生物这一领域。

In [52]:
cytoscapePlot(convertIdentifiers(p, "symbol"), which = "mixed")

Loading data...
Applying default style...
Applying preferred layout...


# 6 Topological pathway analysis

graphite 提供了三种Topological pathway analysis。 更多输出结果细节可以查看相关的R包。 根据说明书，除clipper http://bioconductor.org/packages/release/bioc/html/clipper.html 
外其他所有方法都只适用于需带有蛋白质和代谢物的edges的通路分析。 clipper是唯一可以用来研究代谢组学数据的分析方法。

# 6.1 SPIA

SPIA分析需要将gene-gene 网络转换到相应的格式。 这个转换是由于prepareSPIA()函数来执行，而且必须在runSPIA()之前.

SPIA数据应该保存在当前工作目录下。每次改变数据之后，都需要重新运行prepareSPIA().

SPIA并不包含edge ，并被强制变成SPIA类型。 代谢物之间相互作用（通过代谢物繁殖的边edge) 在graphite用“indirect”被注释。更多信息查看SPIA package。

http://bioconductor.org/packages/release/bioc/html/SPIA.html

In [35]:
library(SPIA)

Loading required package: KEGGgraph

Attaching package: 'KEGGgraph'

The following object is masked from 'package:graphics':

    plot



In [43]:
data(colorectalcancer)
#数据有三个，一个数据框top 两个values 
head(top)

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

,ID,logFC,AveExpr,t,P.Value,adj.P.Val,B,ENTREZ
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
10738,201289_at,5.960206,6.226928,23.94388,1.789221e-17,9.782565e-13,25.40124,3491
18604,209189_at,5.143502,7.487305,17.42995,1.560212e-14,2.843486e-10,21.02120,2353
11143,201694_s_at,4.148081,7.038281,16.46040,5.153117e-14,7.043667e-10,20.14963,1958
10490,201041_s_at,2.429889,9.594413,14.06891,1.293706e-12,1.414667e-08,17.66883,1843
10913,201464_x_at,1.531126,8.221044,10.98634,1.685519e-10,1.151947e-06,13.61189,3725
11463,202014_at,1.429269,5.327647,10.45906,4.274251e-10,2.418771e-06,12.80131,23645


In [44]:
library(hgu133plus2.db)
top$ENTREZ <- mapIds(hgu133plus2.db,
                     top$ID, "ENTREZID", "PROBEID", multiVals = "first")
head(top)

'select()' returned 1:many mapping between keys and columns


,ID,logFC,AveExpr,t,P.Value,adj.P.Val,B,ENTREZ
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
10738,201289_at,5.960206,6.226928,23.94388,1.789221e-17,9.782565e-13,25.40124,3491
18604,209189_at,5.143502,7.487305,17.42995,1.560212e-14,2.843486e-10,21.02120,2353
11143,201694_s_at,4.148081,7.038281,16.46040,5.153117e-14,7.043667e-10,20.14963,1958
10490,201041_s_at,2.429889,9.594413,14.06891,1.293706e-12,1.414667e-08,17.66883,1843
10913,201464_x_at,1.531126,8.221044,10.98634,1.685519e-10,1.151947e-06,13.61189,3725
11463,202014_at,1.429269,5.327647,10.45906,4.274251e-10,2.418771e-06,12.80131,23645


In [45]:
top <- top[!is.na(top$ENTREZ) & !duplicated(top$ENTREZ), ] #删除ENTREZ中的na和重复的数据
top$ENTREZ <- paste("ENTREZID", top$ENTREZ, sep = ":") #给每一个ENTREZ重新编写
tg1 <- top[top$adj.P.Val < 0.05, ] #选择p值小于0.05的
DE_Colorectal = tg1$logFC
names(DE_Colorectal) <- tg1$ENTREZ
ALL_Colorectal <- top$ENTREZ

biocarta <- pathways("hsapiens", "biocarta")[1:10]
biocarta <- convertIdentifiers(biocarta, "ENTREZID")

prepareSPIA(biocarta, "biocartaEx")


Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

In [46]:
res <- runSPIA(de = DE_Colorectal, all = ALL_Colorectal, "biocartaEx")
res[1:5,]


Done pathway 1 : p38 mapk signaling pathway..
Done pathway 2 : regulation of eif-4e and p70s6..
Done pathway 3 : tnfr2 signaling pathway..
Done pathway 4 : melanocyte development and pig..
Done pathway 5 : il-7 signal transduction..
Done pathway 6 : ifn gamma signaling pathway..

,Name,pSize,NDE,pNDE,tA,pPERT,pG,pGFdr,pGFWER,Status
,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,p38 mapk signaling pathway,21,7,0.1213862,-4.6267899,0.676,0.2872276,0.4168242,1,Inhibited
2,tnfr2 signaling pathway,9,1,0.8739131,0.6290229,0.103,0.3067465,0.4168242,1,Activated
3,melanocyte development and pigmentation pathway,12,4,0.2201557,5.9625290,0.420,0.3126182,0.4168242,1,Activated
4,regulation of eif-4e and p70s6 kinase,18,2,0.9101075,0.0000000,1.000,0.9958329,0.9958329,1,Inhibited
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


# 6.2 topologyGSA

topologyGSA 是用graphical模型来测试pathway组成和标注异常部分

graphite, topologyGSA 、中，runTopologyGSA() 可以用来分析单个pathway或pathway list

In [2]:
library(topologyGSA)
data(examples)
#包含三个，一个是dag_bcell,另连个是y1、y2

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windowsWarning message:
"package 'topologyGSA' was built under R version 3.5.3"No methods found in package 'GenomicRanges' for request: 'c' when loading 'qpgraph'


In [3]:
class(dag_bcell)
str(dag_bcell)

[1] "graphNEL"
attr(,"package")
[1] "graph"

Formal class 'graphNEL' [package "graph"] with 6 slots
  ..@ nodes     : chr [1:35] "SHP1" "CD72" "CD22" "SYK" ...
  ..@ edgeL     :List of 35
  .. ..$ SHP1   :List of 1
  .. .. ..$ edges: int [1:2] 4 8
  .. ..$ CD72   :List of 1
  .. .. ..$ edges: int 1
  .. ..$ CD22   :List of 1
  .. .. ..$ edges: int 1
  .. ..$ SYK    :List of 1
  .. .. ..$ edges: int [1:3] 9 12 29
  .. ..$ LYN    :List of 1
  .. .. ..$ edges: int [1:5] 4 6 7 8 11
  .. ..$ CD79   :List of 1
  .. .. ..$ edges: num(0) 
  .. ..$ CD19   :List of 1
  .. .. ..$ edges: num(0) 
  .. ..$ BTK    :List of 1
  .. .. ..$ edges: int [1:2] 9 12
  .. ..$ BLNK   :List of 1
  .. .. ..$ edges: int [1:3] 10 12 15
  .. ..$ VAV    :List of 1
  .. .. ..$ edges: int 11
  .. ..$ RAC    :List of 1
  .. .. ..$ edges: num(0) 
  .. ..$ PLCG2  :List of 1
  .. .. ..$ edges: int [1:3] 13 17 23
  .. ..$ CaN    :List of 1
  .. .. ..$ edges: int 14
  .. ..$ NFAT   :List of 1
  .. .. ..$ edges: num(0) 
  .. ..$ GRB2   :List of 1
  .. .. ..$ edges: int

In [4]:
colnames(y1) <- paste("SYMBOL", colnames(y1), sep = ":")
colnames(y2) <- paste("SYMBOL", colnames(y2), sep = ":")
head(y1)
head(y2)

SYMBOL:CD22,SYMBOL:CD72,SYMBOL:SHP1,SYMBOL:SYK,SYMBOL:CD79,SYMBOL:LYN,SYMBOL:BTK,SYMBOL:BLNK,SYMBOL:VAV,SYMBOL:RAC,...,SYMBOL:NFKB,SYMBOL:FcgRIIB,SYMBOL:SHIP,SYMBOL:PI3K,SYMBOL:AKT,SYMBOL:GSK3B,SYMBOL:CD19,SYMBOL:CD21,SYMBOL:CD81,SYMBOL:LEU13
8.89,5.99,5.94,5.43,11.74,6.35,5.91,8.62,6.64,5.29,...,10.22,5.80,9.53,7.11,7.28,5.37,9.93,4.51,10.94,10.02
8.39,7.05,6.41,5.70,12.77,5.86,6.11,9.29,6.71,6.22,...,10.67,5.82,9.38,6.50,6.99,6.06,9.33,3.88,10.50,11.15
8.58,5.75,5.98,5.64,11.45,5.57,5.61,9.35,6.42,5.80,...,10.64,5.55,9.39,6.24,6.73,5.35,10.53,4.26,10.98,10.01
8.58,6.45,5.83,5.75,12.82,6.53,5.76,7.89,6.76,5.55,...,10.29,5.98,9.56,7.60,7.18,5.81,9.57,4.47,11.51,9.86
9.10,6.28,6.87,5.82,15.18,5.55,6.92,8.23,6.97,7.79,...,9.51,5.67,9.43,5.08,9.27,5.39,10.62,4.02,10.93,10.41
8.83,6.83,5.50,5.95,10.68,5.53,6.07,9.12,6.74,5.31,...,9.67,5.64,10.21,5.78,7.00,5.80,9.57,5.06,10.43,9.09


SYMBOL:CD22,SYMBOL:CD72,SYMBOL:SHP1,SYMBOL:SYK,SYMBOL:CD79,SYMBOL:LYN,SYMBOL:BTK,SYMBOL:BLNK,SYMBOL:VAV,SYMBOL:RAC,...,SYMBOL:NFKB,SYMBOL:FcgRIIB,SYMBOL:SHIP,SYMBOL:PI3K,SYMBOL:AKT,SYMBOL:GSK3B,SYMBOL:CD19,SYMBOL:CD21,SYMBOL:CD81,SYMBOL:LEU13
9.69,7.18,7.76,7.33,13.85,6.94,5.94,7.88,7.08,8.61,...,9.82,5.98,8.97,5.92,8.28,5.45,10.12,4.22,10.54,10.79
8.44,5.95,5.26,4.96,11.40,5.28,6.75,9.08,6.23,5.95,...,9.88,5.60,10.27,6.70,7.27,5.45,10.00,3.56,11.04,10.71
10.19,5.96,7.44,5.29,10.81,7.14,4.86,6.46,6.96,6.03,...,10.12,5.71,9.09,5.13,7.52,5.22,9.19,3.63,9.98,9.84
8.74,7.15,7.67,7.40,14.42,6.04,7.13,7.95,6.96,8.51,...,9.55,5.78,9.47,5.58,8.88,5.39,10.37,4.10,10.33,9.69
9.00,6.10,5.79,5.76,10.01,6.40,7.08,7.14,6.42,6.07,...,9.91,5.60,8.76,6.01,7.14,5.49,9.13,3.67,9.86,10.33
7.76,8.58,5.42,4.90,10.51,5.72,6.57,5.72,6.81,5.35,...,9.33,5.97,9.61,6.58,7.16,5.45,9.74,3.68,10.14,7.79


In [5]:
kegg <- pathways("hsapiens", "kegg")
p <- convertIdentifiers(kegg[["Fc epsilon RI signaling pathway"]], "SYMBOL")
runTopologyGSA(p, "var", y1, y2, 0.05)

'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns
'select()' returned 1:1 mapping between keys and columns



        Pathway Variance Test

data: exp1, exp2 and g

lambda = 26.02199, df = 10, p-value = 0.003710726, equal variances: TRUE


In [7]:
head(y2)

SYMBOL:CD22,SYMBOL:CD72,SYMBOL:SHP1,SYMBOL:SYK,SYMBOL:CD79,SYMBOL:LYN,SYMBOL:BTK,SYMBOL:BLNK,SYMBOL:VAV,SYMBOL:RAC,...,SYMBOL:NFKB,SYMBOL:FcgRIIB,SYMBOL:SHIP,SYMBOL:PI3K,SYMBOL:AKT,SYMBOL:GSK3B,SYMBOL:CD19,SYMBOL:CD21,SYMBOL:CD81,SYMBOL:LEU13
9.69,7.18,7.76,7.33,13.85,6.94,5.94,7.88,7.08,8.61,...,9.82,5.98,8.97,5.92,8.28,5.45,10.12,4.22,10.54,10.79
8.44,5.95,5.26,4.96,11.40,5.28,6.75,9.08,6.23,5.95,...,9.88,5.60,10.27,6.70,7.27,5.45,10.00,3.56,11.04,10.71
10.19,5.96,7.44,5.29,10.81,7.14,4.86,6.46,6.96,6.03,...,10.12,5.71,9.09,5.13,7.52,5.22,9.19,3.63,9.98,9.84
8.74,7.15,7.67,7.40,14.42,6.04,7.13,7.95,6.96,8.51,...,9.55,5.78,9.47,5.58,8.88,5.39,10.37,4.10,10.33,9.69
9.00,6.10,5.79,5.76,10.01,6.40,7.08,7.14,6.42,6.07,...,9.91,5.60,8.76,6.01,7.14,5.49,9.13,3.67,9.86,10.33
7.76,8.58,5.42,4.90,10.51,5.72,6.57,5.72,6.81,5.35,...,9.33,5.97,9.61,6.58,7.16,5.45,9.74,3.68,10.14,7.79


# clipper

clipper 也是一个拓扑结构分析的R包。 

In [53]:
library(ALL)
library(a4Preproc)
library(clipper)

data(ALL)
pheno <- as(phenoData(ALL), "data.frame")
samples <- unlist(lapply(c("NEG", "BCR/ABL"), function(t) {
  which(grepl("^B\\d*", pheno$BT) & (pheno$mol.biol == t))[1:10]
}))
classes <- c(rep(1,10), rep(2,10))

expr <- exprs(ALL)[,samples]
rownames(expr) <- paste("ENTREZID",
                        featureData(addGeneInfo(ALL))$ENTREZID,
                        sep = ":")

k <- as.list(pathways("hsapiens", "kegg"))
selected <- k[c("Chronic myeloid leukemia", 
                "Bladder cancer", 
                "Cytosolic DNA-sensing pathway")]

clipped <- runClipper(selected, expr, classes, "mean", pathThr = 0.1)
resClip <- do.call(rbind, clipped$results)
resClip[, c("startIdx", "endIdx", "maxIdx", "lenght",
            "maxScore", "aScore", "involvedGenes")]


Loading required package: Matrix

Attaching package: 'Matrix'

The following object is masked from 'package:S4Vectors':

    expand

Loading required package: graph
Loading required package: hgu95av2.db



NULL

In [55]:
clipped


$results
named list()

$warnings
named list()

$errors
                                                                           Chronic myeloid leukemia 
"Error in .qpFastIPF(vv, clqlst, tol, verbose): Lapack routine dgesv: system is exactly singular\n"

In [3]:
if (require(clipper) & require(ALL) & require(a4Preproc)) {
  data(ALL)
  pheno <- as(phenoData(ALL), "data.frame")
  samples <- unlist(lapply(c("NEG", "BCR/ABL"), function(t) {
    which(grepl("^B\\d*", pheno$BT) & (pheno$mol.biol == t))[1:10]
  }))
  classes <- c(rep(1,10), rep(2,10))

  expr <- exprs(ALL)[,samples]
  rownames(expr) <- paste("ENTREZID", featureData(addGeneInfo(ALL))$ENTREZID,
                          sep = ":")

  k <- as.list(pathways("hsapiens", "kegg"))
  selected <- k[c("Bladder cancer", "Hippo signaling pathway - multiple species")]

  runClipper(selected, expr, classes, "mean", pathThr = 0.1)
}

,startIdx,endIdx,maxIdx,lenght,maxScore,aScore,activation,impact,involvedCliques,cliqueOnPath,involvedGenes,pathGenes
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1;3,1,3,2,3,5.83818294928244,2.91909147464122,0.281721182135002,0.5,"1,2","1,2,3",ENTREZID:10413;ENTREZID:25937;ENTREZID:5058;ENTREZID:7003;ENTREZID:10413;ENTREZID:25937;ENTREZID:5058;ENTREZID:7004,"ENTREZID:10413;ENTREZID:25937;ENTREZID:5058;ENTREZID:7003,ENTREZID:10413;ENTREZID:25937;ENTREZID:5058;ENTREZID:7004,ENTREZID:10413;ENTREZID:25937;ENTREZID:5058;ENTREZID:7005"


In [5]:
class(expr)
head(expr)

[1] "matrix"

,01010,04007,04008,04010,04016,06002,08012,08024,09017,12019,01005,03002,08001,08011,09008,11005,12006,12007,12012,12026
ENTREZID:5595,7.479445,7.905312,7.065914,7.474537,7.536119,7.183331,7.824284,7.879988,7.756734,7.652719,7.597323,7.567593,7.735545,7.591498,7.891793,7.640012,7.759599,7.678636,7.464285,7.501591
ENTREZID:7075,4.932537,4.844565,5.147762,5.122518,5.016132,5.288943,4.685951,4.830464,4.987595,5.175609,5.046194,4.799294,4.633217,4.583148,5.999496,4.967288,4.770481,5.456332,4.785863,5.188992
ENTREZID:1557,4.208155,3.416923,3.945869,4.150506,3.576360,3.900935,3.902139,3.862914,4.048901,3.932360,3.900466,3.886169,3.630190,3.609112,4.001606,3.796550,3.912707,3.870893,3.930832,4.188444
ENTREZID:643,6.169024,5.687997,6.208061,6.292713,5.665991,5.842326,5.762857,6.079410,6.097900,6.194623,5.903856,5.860459,5.875375,5.733157,5.832952,6.094379,6.235795,5.971466,6.037364,6.231228
ENTREZID:643,5.912780,5.615210,5.923487,6.046607,5.738218,5.994515,5.679899,6.057632,6.210092,5.969027,5.925260,5.893209,5.748350,5.922568,5.717497,5.751805,5.883340,5.918456,5.725421,6.357476
ENTREZID:1843,10.428299,9.983809,10.063484,10.662059,11.269115,8.812869,8.227970,7.667445,10.015466,10.243610,8.570990,9.616713,10.165159,9.381072,10.206353,9.358516,8.824348,9.262478,7.232927,7.808452


In [7]:
classes

[1] 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2

In [9]:
Clipper_result <- runClipper(selected, expr, classes, "mean", pathThr = 0.1)

In [10]:
Clipper_result$results

,startIdx,endIdx,maxIdx,lenght,maxScore,aScore,activation,impact,involvedCliques,cliqueOnPath,involvedGenes,pathGenes
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1;3,1,3,2,3,5.83818294928244,2.91909147464122,0.281721182135002,0.5,"1,2","1,2,3",ENTREZID:10413;ENTREZID:25937;ENTREZID:5058;ENTREZID:7003;ENTREZID:10413;ENTREZID:25937;ENTREZID:5058;ENTREZID:7004,"ENTREZID:10413;ENTREZID:25937;ENTREZID:5058;ENTREZID:7003,ENTREZID:10413;ENTREZID:25937;ENTREZID:5058;ENTREZID:7004,ENTREZID:10413;ENTREZID:25937;ENTREZID:5058;ENTREZID:7005"
